In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

# Load data
blends_data = pd.read_csv('/Users/huiyisang/Desktop/blending_blends.csv')
materials_data = pd.read_csv('/Users/huiyisang/Desktop/blending_materials.csv')

# Get dimensions
num_blends = blends_data.shape[0]  # 50 blends
num_materials = materials_data.shape[0]  # 100 materials

# Extract data from files
demands = blends_data['demand'].values
quality_min = blends_data['quality_min'].values
quality_max = blends_data['quality_max'].values

availability = materials_data['availability'].values
costs = materials_data['cost'].values
p_max = materials_data['p_max'].values

# Extract quality contribution matrix (50x100)
#quality_columns = [f'quality_{j+1}' for j in range(num_materials)]
#quality_contribution = blends_data[quality_columns].values
quality_contribution = blends_data.iloc[:, :-3].values 

# Create model
model = gp.Model("EcoClean_Blending")

# Create variables - x[i,j] is the amount of material j used in blend i
x = model.addVars(num_blends, num_materials, vtype=GRB.CONTINUOUS, name="x")

# Set objective - minimize total cost
objective = gp.quicksum(x[i, j] * costs[j] for i in range(num_blends) for j in range(num_materials))
model.setObjective(objective, GRB.MINIMIZE)

# Constraint 1: Meet demand for each blend
for i in range(num_blends):
    model.addConstr(gp.quicksum(x[i, j] for j in range(num_materials)) == demands[i], 
                    f"demand_constraint_{i}")

# Constraint 2: Raw material availability
for j in range(num_materials):
    model.addConstr(gp.quicksum(x[i, j] for i in range(num_blends)) <= availability[j], 
                    f"availability_constraint_{j}")

# Constraint 3: Quality constraints for each blend
for i in range(num_blends):
    # Calculate the average quality for blend i
    blend_quality = gp.quicksum(x[i, j] * quality_contribution[i, j] 
                              for j in range(num_materials)) / demands[i]
    
    # Enforce quality bounds
    model.addConstr(blend_quality >= quality_min[i], f"min_quality_constraint_{i}")
    model.addConstr(blend_quality <= quality_max[i], f"max_quality_constraint_{i}")

# Constraint 4: Maximum proportion constraint for each material in each blend
for i in range(num_blends):
    for j in range(num_materials):
        model.addConstr(x[i, j] <= p_max[j] * gp.quicksum(x[i, k] for k in range(num_materials)), 
                       f"proportion_constraint_{i}_{j}")

# Optimize the model
model.optimize()

# Print the optimal objective value
if model.status == GRB.OPTIMAL:
    print(f"Minimum production cost: ${model.objVal:.2f}")
else:
    print("No optimal solution found.")

Set parameter Username
Set parameter LicenseID to value 2610034
Academic license - for non-commercial use only - expires 2026-01-14
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5250 rows, 5000 columns and 520000 nonzeros
Model fingerprint: 0x761ad8dc
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 1e+04]
Presolve removed 50 rows and 0 columns
Presolve time: 0.05s
Presolved: 5200 rows, 5050 columns, 515050 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...


Barrier performed 0 iterations in 0.07 seconds (0.19 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Iteration    Objective       Primal Inf.    Dual Inf.      Time
  